In [12]:
import pandas as pd
import numpy as np
from utility_functions import my_confusion_matrix, my_classification_scores, my_dataframe_split

In [9]:
# Load the dataset
insurance_train_path = 'data/train.csv'
insurance_train_df = pd.read_csv(insurance_train_path,index_col=None)
insurance_train_df.columns= insurance_train_df.columns.str.lower()
insurance_train_df.columns = insurance_train_df.columns.str.replace(' ','_')

# Map 'Yes' to 1 and 'No' to 0
insurance_train_df['vehicle_damage'] = insurance_train_df['vehicle_damage'].map({'Yes': 1, 'No': 0})

In [10]:
train_df, validation_df, test_df = my_dataframe_split(my_test_df,0.7,0.15,0.15)